##  Build a simple CNN model for classifying dog/cat  

Author: Anh Trung Tra   
Email: tratrunganh001@gmail.com

In [6]:
import tensorflow as tf
keras = tf.keras
%matplotlib inline

In [32]:
# Create a dictionary describing the features.  
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto, size, is_augmented=True):
    """ TODO: Update description!
    """
    parsed_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image_string = parsed_example['image_raw']
    label = parsed_example['label']
    
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32) # convert to float values in [0, 1]
    
    if is_augmented: # augmented here!
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.clip_by_value(image, 0.0, 1.0)
        
    # resize image to fit the input size
    resized_image = tf.image.resize_with_pad(image, size, size)

    return resized_image, label

train_dataset = (tf.data.TFRecordDataset('data/PetImages/train_14898.tfrecord')
                     .map(lambda ep: _parse_function(ep, 28, True), num_parallel_calls=4)
                     .shuffle(1000)
                     .repeat()
                     .batch(64)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

val_dataset = (tf.data.TFRecordDataset('data/PetImages/val_4968.tfrecord')
                     .map(lambda ep: _parse_function(ep, 28, False), num_parallel_calls=4)
                     .batch(64)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

test_dataset = (tf.data.TFRecordDataset('data/PetImages/test_4958.tfrecord')
                     .map(lambda ep: _parse_function(ep, 28, False), num_parallel_calls=4)
                     .batch(64)
                     .prefetch(1)  # make sure you always have one batch ready to serve
                )

In [30]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(16, kernel_size=3, activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu'))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu'))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))
model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 64)              

In [31]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit_generator(train_dataset, steps_per_epoch=100, epochs=10, validation_data=val_dataset)
model.evaluate_generator(test_dataset)

Epoch 1/10
100/100 [==============================] - 27s 275ms/step - loss: 0.6899 - accuracy: 0.5431 - val_loss: 0.6838 - val_accuracy: 0.5352
Epoch 2/10
100/100 [==============================] - 23s 228ms/step - loss: 0.6624 - accuracy: 0.6047 - val_loss: 0.6426 - val_accuracy: 0.6437
Epoch 3/10
100/100 [==============================] - 24s 238ms/step - loss: 0.6418 - accuracy: 0.6303 - val_loss: 0.6268 - val_accuracy: 0.6564
Epoch 4/10
100/100 [==============================] - 24s 242ms/step - loss: 0.6251 - accuracy: 0.6509 - val_loss: 0.6219 - val_accuracy: 0.6645
Epoch 5/10
100/100 [==============================] - 26s 262ms/step - loss: 0.6213 - accuracy: 0.6569 - val_loss: 0.6121 - val_accuracy: 0.6787
Epoch 6/10
100/100 [==============================] - 26s 262ms/step - loss: 0.6055 - accuracy: 0.6716 - val_loss: 0.5946 - val_accuracy: 0.6886
Epoch 7/10
100/100 [==============================] - 25s 248ms/step - loss: 0.5928 - accuracy: 0.6897 - val_loss: 0.5948 - val_ac

[0.5584027171134949, 0.71762806]